In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 14

## Sampling from a Population

In [ ]:
# Load a table of United Airlines flights
united = Table.read_table('data/united.csv')
united

This table represents our *population*, consisting of almost 14,000 individuals. Let's use a histogram to visualize the probability distribution of the `Delay` variable:

In [ ]:
united_bins = np.arange(-20, 201, 5)
united.hist('Delay', bins = united_bins, unit='min')
plots.title('Probability Distribution of Delay');

**Review Question:** does the histogram indicate that most flights are early or that most flights are late? (*Hint:* we are using equally-sized bins with a width of 5 minutes.)

In [ ]:
# ...

Using the `sample` table method, we can draw simple random samples of the population contained in the `united` table. (Remember that a *simple random sample* is one in which all equally-sized subsets of the population have an equal chance of showing up in the sample.) By default, `sample` sets `with_replacement=True`, so that the simple random sample occurs with replacement.

Once our sample is drawn, we will visualize the *empirical distributions* of the `Delay` variable, using a histogram with the same bins.

In [ ]:
# Empirical distribution with a sample size of 10
united.sample(10).hist('Delay', bins = united_bins, unit='min')
plots.title('Empirical Distribution of Delay (n=10)');

In [ ]:
# Empirical distribution with a sample size of 100
united.sample(100).hist('Delay', bins = united_bins, unit='min')
plots.title('Empirical Distribution of Delay (n=100)');

In [ ]:
# Empirical distribution with a sample size of 1000
united.sample(1000).hist('Delay', bins = united_bins, unit='min')
plots.title('Empirical Distribution of Delay (n=1000)');

Due to the law of large numbers (the book refers to this as the "law of averages"), it is very likely that the empirical distribution will get closer and closer to the underlying probability distribution as the sample gets larger.

## Inference

Let's load the class data survey and select the `Pets` column, which contains the number of pets that each student in the class has. In this example, our population consists of respondents to the class data survey.

In [ ]:
class_data = Table.read_table('./data/cmpsc5a-classdata-w23.csv')
class_population = class_data.select('Pets').where('Pets', are.above_or_equal_to(0))
class_population

Somewhere in this table is the largest value of `Pets`---don't look for it just yet! This largest value is an example of a *parameter* of the population.

What if only 10 students had responded to the survey? Using the `sample` method, we will choose a simple random sample (without replacement) from the `class_population` table:

In [ ]:
class_sample = class_population.sample(10, with_replacement=False)
class_sample

**Question:** think of a reasonable *statistic* that we could use to estimate the largest value of `Pets` in the population from the data in the `class_sample` table. Calculate the value of that statistic.

In [ ]:
pets_max_estimate = ...

In [ ]:
pets_max_estimate = max(class_sample.column('Pets'))
pets_max_estimate

How accurate was this estimate? Compare it with the true value of the parameter below:

In [ ]:
max(class_population.column('Pets'))

## Sampling Distribution

A reasonable choice for a statistic to estimate the parameter `max(class_population.column('Pets'))` is to use the maximum value of `Pets` in `class_sample`:

In [ ]:
pets_max_estimate = max(class_sample.column('Pets'))
pets_max_estimate

But this value depends on the sample `class_sample`, which is a random sample. Try re-drawing the class sample and re-calculating the `pets_max_estimate` statistic several times. You should see a few different values.

In [ ]:
# Randomly re-draw the sample
class_sample = class_population.sample(10, with_replacement=False)

# Re-calculate the statistic
pets_max_estimate = max(class_sample.column('Pets'))
pets_max_estimate

Since the statistic `pets_max_estimate` is a random quantity, it is associated with a probability distribution, which depends on the underlying population `class_population` and our strategy for selecting the sample (e.g., a simple random sample without replacement).

This statistic is simple enough that we can actually calculate the sampling distribution by hand! Let's look at the probability distribution for `Pets` from the full population, using the `group` method:

In [ ]:
print('Population size:', class_population.num_rows)
pet_count_frequencies = class_population.group('Pets')
pet_count_frequencies

**Question 1:** if we randomly select one survey respondent, where each respondent is equally likely, what is the probability that the respondent will not have a pet?

In [ ]:
# ...

**Question 2:** if we randomly select two survey respondents *without replacement*, where each respondent is equally likely, what is the probability that neither of these two respondents will have a pet?

In [ ]:
# ...

**Question 3 (hard):** if we randomly select 10 survey respondents without replacement, where each respondent is equally likely, what is the probability that none of the 10 respondents will have a pet?

You may find it helpful to write a `for` loop to calculate the number.

In [ ]:
# ...

In [ ]:
remaining_population_size = 81
remaining_petless_respondents = 30
prob_no_pets = 1

for i in np.arange(10):
    
    # Calculate the probability that the next respondent has no pets,
    # given that all of the previous respondents didn't have pets either
    prob_respondent_i_is_petless = remaining_petless_respondents / remaining_population_size
    
    # Update the probability of prob_no_pets using the multiplication rule
    prob_no_pets = prob_no_pets * prob_respondent_i_is_petless
    
    # Update the remaining population sizes for the next round
    remaining_population_size = remaining_population_size - 1
    remaining_petless_respondents = remaining_petless_respondents - 1
    
print('Probability that none of 10 respondents have pets:', prob_no_pets)

**Question 4:** given the information above, what is the probability that `pets_max_estimate == 0`?

In [ ]:
# ...

We can calculate the other probabilities P(`pets_max_estimate == 1`), P(`pets_max_estimate == 2`), and so on in a similar manner. In the interest of finishing lecture on time, we won't go into details, but the function `prob_sample_max_is_x` below will calculate these probabilities for you, using the help of a function `prob_sample_max_is_at_most_x`. 

If you want a take-home challenge, try to understand what the two functions are doing, and how they use the basic probability rules to calculate the probability that `pets_max_estimate == x`!

In [ ]:
def prob_sample_max_is_x(x, n):
    """
    Calculate the probability that the maximum value of pets in a simple random sample
    (without replacement) of n respondents is equal to x.
    """
 
    # If x > 0, then by the addition rule, P(max <= x) == P(max == x) + P(max < x).
    # Therefore, P(max == x) = P(max <= x) - P(max < x).
    # Since x is an integer, P(max < x) = P(max <= x - 1).
    
    # We define a "helper function" prob_sample_max_is_at_most_x to calculate P(max <= x).
    
    if x == 0:
        return prob_sample_max_is_at_most_x(0, n)
    elif x > 0:
        return prob_sample_max_is_at_most_x(x, n) - prob_sample_max_is_at_most_x(x - 1, n)
    
    
def prob_sample_max_is_at_most_x(x, n):
    """
    Calculate the probability that the maximum value of pets in a simple random sample
    (without replacement) of n respondents is <= x. 
    
    E.g., prob_sample_max_is_at_most_x(0, 10) should return the same value we got for prob_no_pets above.
    
    This is an example of a "helper function."
    When we're coding a complicated function, it's often helpful to define extra functions
    that the original function can call multiple times.
    """
    
    remaining_population_size = 81
    
    # Calculate how many respondents have at most x pets
    remaining_respondents_with_at_most_x_pets = pet_count_frequencies.where(
        'Pets', are.below_or_equal_to(x)).column('count').sum()
    
    prob_pets_at_most_x = 1
    
    for i in np.arange(n):
        
        # Calculate the probability that the next respondent has at most x pets,
        # given that all of the previous respondents had at most x pets as well
        prob_respondent_i_at_most_x = remaining_respondents_with_at_most_x_pets / remaining_population_size
        
        # Update the probability of prob_pets_at_most_x using the multiplication rule
        prob_pets_at_most_x = prob_pets_at_most_x * prob_respondent_i_at_most_x
        
        # Update the remaining population sizes for the next round
        remaining_population_size = remaining_population_size - 1
        remaining_respondents_with_at_most_x_pets = remaining_respondents_with_at_most_x_pets - 1
        
    return prob_pets_at_most_x

Let's create a table indicating numbers of pets between 0 and 6, and use the `apply` method to calculate the probability that `pets_max_estimate` takes on each of these values, given a sample size of 10.

In [ ]:
sampling_dist = Table().with_columns(
    'Pets', np.arange(0, 7),
    'sample size', 10)
sampling_dist = sampling_dist.with_column(
    'Probability', sampling_dist.apply(prob_sample_max_is_x, 'Pets', 'sample size'))
sampling_dist

In [ ]:
# Visualize the sampling distribution using a bar chart
sampling_dist.bar('Pets', 'Probability')
plots.title('Sampling Distribution of the max statistic (n=10)');

**Question:** given a sample size of 10, does the max statistic provide a good estimate for the maximum value of `Pets` in the original population?

In [ ]:
# ...

What if we triple the sample size?

In [ ]:
# Re-compute the table indicating the sampling distribution,
# but now use a sample size of n = 30
sampling_dist = Table().with_columns(
    'Pets', np.arange(0, 7),
    'sample size', 30)
sampling_dist = sampling_dist.with_column(
    'Probability', sampling_dist.apply(prob_sample_max_is_x, 'Pets', 'sample size'))

# Visualize the sampling distribution using a bar chart
sampling_dist.bar('Pets', 'Probability')
plots.title('Sampling Distribution of the max statistic (n=30)');

What if the sample includes the entire population?

In [ ]:
# Re-compute the table indicating the sampling distribution,
# but now use a sample size of n = 81
sampling_dist = Table().with_columns(
    'Pets', np.arange(0, 7),
    'sample size', 81)
sampling_dist = sampling_dist.with_column(
    'Probability', sampling_dist.apply(prob_sample_max_is_x, 'Pets', 'sample size'))

# Visualize the sampling distribution using a bar chart
sampling_dist.bar('Pets', 'Probability')
plots.title('Sampling Distribution of the max statistic (n=81)');

**Question:** why is 5 the only possible value that the max statistic can have when the sample size is 81? (Remember we are sampling without replacement!)

In [ ]:
# ...

## Statistics

Let's return to the table of United flights. One example of a parameter in this population is the median delay:

In [ ]:
# (Population) Parameter
np.median(united.column('Delay'))

In order to estimate the paramter from a sample, we might reasonably calculate the median delay from that sample:

In [ ]:
# (Sample) Statistic
np.median(united.sample(10).column('Delay'))

In [ ]:
# (Sample) Statistic
np.median(united.sample(100).column('Delay'))

Is this statistic a good estimator for the median parameter? In order to find out, we'll need to know--or at least approximate--the sampling distribution. The math on this one is very complicated, so instead of calculating the probability distribution exactly, we will approximate it by constructing an empirical distribution of the statistic.

In [ ]:
# For 2000 simulations: create a simple random sample with 10 rows from the table, 
# then calculate the median delay statistic. Save the values of these statistics
# to an array sample_medians.

num_simulations = 2000
sample_medians = make_array()

for i in np.arange(num_simulations):
    new_median = np.median(united.sample(10).column('Delay'))
    sample_medians = np.append(sample_medians, new_median)

In [ ]:
# Plot the empirical distribution with a histogram
median_bins = np.arange(-5, 30)
Table().with_column('Sample medians (size=10)', sample_medians).hist(bins=median_bins)

What if we increase the sample size from 10 to 1000?

In [ ]:
sample_medians = make_array()

# This loop will take a moment to finish!
for i in np.arange(num_simulations):
    new_median = np.median(united.sample(1000).column('Delay'))
    sample_medians = np.append(sample_medians, new_median)

In [ ]:
# Plot the empirical distribution with a histogram
Table().with_column('Sample medians (size=1K)', sample_medians).hist(bins=median_bins)

We can overlay several histograms into one plot, to get an even clearer idea of what happens to the (empirical) sampling distribution as the sample size becomes larger:

In [ ]:
sample_medians_10 = make_array()
sample_medians_100 = make_array()
sample_medians_1000 = make_array()

num_simulations = 2000

# Perform the simulations
# This loop will take a moment to finish!
for i in np.arange(num_simulations):
    new_median_10 = np.median(united.sample(10).column('Delay'))
    sample_medians_10 = np.append(sample_medians_10, new_median_10)
    new_median_100 = np.median(united.sample(100).column('Delay'))
    sample_medians_100 = np.append(sample_medians_100, new_median_100)
    new_median_1000 = np.median(united.sample(1000).column('Delay'))
    sample_medians_1000 = np.append(sample_medians_1000, new_median_1000)
    
# Plot the histogram
sample_medians = Table().with_columns('Size 10', sample_medians_10, 
                                  'Size 100', sample_medians_100,
                                  'Size 1000', sample_medians_1000)
sample_medians.hist(bins=median_bins)

## Swain vs. Alabama ##

We want to simulate the model where the jury panel is selected by a simple random sample of the elgible population. If this model is correct, how do we simulate the number of Black men on the jury panel?

The `np.random.choice` function has an optional argument `p`, which takes an array of floats indicating the probability of each element in the array.
For example, the following statement selects the string `"Black"` with probability 0.26, and `"Not Black"` with probability 0.74:

In [ ]:
np.random.choice(make_array('Black', 'Not Black'), p=make_array(0.26, 0.74))

Using this function, we can use `for` and `if` control statements to simulate a truly random jury selection, and count the number of Black men on each simulated panel.

In [ ]:
def sample_number_black_members(panel_size, p):
    """
    Simulate the selection of a jury panel of size panel_size,
    from an eligible population where the proportion of Black men is p.
    Return the numbe of Black men on the panel.
    """
    
    n_black_members = 0
    
    for i in np.arange(panel_size):
        
        # Decide if jury member i is Black with probability p
        race = np.random.choice(make_array('Black', 'Not Black'), p=make_array(p, 1-p))
        if race == 'Black':
            n_black_members = n_black_members + 1
            
    return n_black_members

In [ ]:
sample_number_black_members(100, 0.26)

Let's simulate 1,000 truly random jury panels, and plot the empirical distribution of the number of Black men in these panels.

In [ ]:
n_black_members_per_panel = make_array()

for i in np.arange(1000):  
    n_black_members_per_panel = np.append(
        n_black_members_per_panel,
        sample_number_black_members(100, 0.26))

In [ ]:
Table().with_column('Number of Black Men on Panel of 100', panels).hist(bins=np.arange(5.5,40.))

**Question:** Is this model consistent with the data that Swain's jury panel had 8 Black men?

In [ ]:
# ...

Mathematically, it is possible to calculate a 0.0005% chance that Swain's panel would have 8 or fewer Black members, if eligble members were truly selected with equal probability. (This is an example of a "Binomial distribution," which you can learn about in PSTAT 120a!)

## Mendel and Pea Flowers ##

In [ ]:
## Mendel had 929 plants, of which 709 had purple flowers
observed_purples = 709 / 929
observed_purples

Instead of using a `for` loop to simulate 929 plants independently, we can use the function `sample_proportions` to do it for us---and faster!

In [ ]:
sample_proportions?

In [ ]:
predicted_proportions = make_array(.75, .25)
sample_proportions(929, predicted_proportions)

In [ ]:
purples = make_array()

for i in np.arange(10000):
    new_purple = sample_proportions(929, predicted_proportions).item(0) * 100
    purples = np.append(purples, new_purple)

In [ ]:
Table().with_column('Percent of purple flowers in sample of 929', purples).hist()

Is this model consistent with Mendel's real-world data that 76.3% of the plants had purple flowers?